In [1]:
import pandas as pd
import numpy as np
import warnings
import re
import os
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

In [2]:
# подгрузка всех файлов без modes2017
dr = 'modes'
file_names = os.listdir(dr)

# файлы в словарь
mode_files = dict()
for name in file_names[1:]:
    mode_files[name] = pd.read_excel(dr+'\\'+name,skiprows=1)
    print(name)
# mode_files.keys()

сводка 1 и 2 квартал 2017_v2.xls
сводка 3 квартал 2017.xls
сводка декабрь 2017_v2.xls
сводка ноябрь 2017_v2.xls
сводка октябрь 2017_v2.xls
сводная 2012_v2.xls
сводная 2013_v2.xls
сводная 2014_v2.xls
сводная 2015_v2.xls
сводная 2016_v2.xls


In [3]:
# Удаление лишнего столбца с комментариями из файла
del mode_files['сводная 2014_v2.xls']['Unnamed: 42']

In [4]:
# Сохраняем столбец с номером партии в переменную batch_number_list 
def rep_part(ls):
    for x in ls:
        x = str(x)
        x = x.replace('.0', '')
    return ls

batch_number_list = []
for mode in mode_files.values():
    for x in mode['№ партии.1']:
        for ch in ['.0']:
            batch_number_list.append(str(x).replace(ch, ''))

batch_number_list = rep_part(batch_number_list)
len(batch_number_list)

129559

In [5]:
# Сохраняем заголовки
header = mode_files['сводка 1 и 2 квартал 2017_v2.xls'].columns
# убираем заголовки
tmp = dict()
for name, mode in zip(mode_files.keys(), mode_files.values()):
    mode.columns = range(mode.shape[1])
    tmp[name] = mode
mode_files = tmp

# убираем последние пять столбцов из всех таблиц
for mode in mode_files.values():
    mode.drop(mode.columns[37:43], axis=1, inplace=True)


In [6]:
# выводим размерности всех файлов
for i in mode_files.values():
    print(i.shape)

(9682, 37)
(5436, 37)
(2437, 37)
(2170, 37)
(2101, 37)
(19689, 37)
(19991, 37)
(18277, 37)
(25074, 37)
(24702, 37)


In [7]:
df_modes = pd.DataFrame()
# слияние всех режимов
for df in mode_files.values():
    df_modes = pd.concat([df_modes,df])

df_modes.shape

(129559, 37)

In [8]:
# Возвращаем заголовки и номера партий, удаляем партии без номеров
len(header)
df_modes.columns
df_modes.columns = header[:37]
df_modes['№ партии'] = batch_number_list
df_modes = df_modes[df_modes['№ партии'] != 'nan']
df_modes.shape

(129086, 37)

In [9]:
#список усредняемых столбцов
l1 = [u'1 зона фактическая температура закалка',
        u'3 зона фактическая температура закалка',
        u'1 зона фактическая температура отпуск',
        u'3 зона фактическая температура отпуск',
        u'5 зона фактическая температура отпуск']

In [10]:
# исходная таблица без пакетов
df_modes = df_modes.drop(['№ пакета'], axis=1)

In [11]:
# Удаляем партии без номера плавки
df_modes = df_modes[df_modes['плавка'] != 'nan']
df_modes.shape

(129086, 36)

In [12]:
# Разносим шаг на закалочной и отпускной печах в разные столбцы и очищаем их
def clean_steps(x):
    x = str(x)
    x = x.replace(u'X', '/')
    x = x.replace(u'x', '/')
    x = x.replace(u'Х', '/')
    x = x.replace(u'х', '/')
    x = x.replace(',', '.')
    x = x.replace('\\', '/')
    x = x.replace('/-', '/0')
    x = x.replace('-/', '0/')
    x = x.replace('***', '0')
    x = x.replace('*', '0')
    x = x.replace('-', '/')
    x = x.replace('nan', '')
    return x

def convert_step0(x):
    return clean_steps(x).split('/')[0]
   
def convert_step1(x):
    try:
        return clean_steps(x).split('/')[1]
    except:
        return x

df_modes = df_modes[df_modes[u'шаг балок закалочная/отпускная печи, сек']!='***']
df_modes[u'шаг балок закалочная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step0)
df_modes[u'шаг балок отпускная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step1)

In [13]:
# Разделяем диаметр и толщину стенки и чистим их
def split_d(x):
    x = str(x)
    x = x.replace(',', '.')
    return re.split('х|Х|x|X', x)[0]

def split_s(x):
    x = str(x)
    x = x.replace(',', '.')
    try:
        return re.split('х|Х|x|X', x)[1]
    except:
        return x
    
def clean_d_s(x):
    x = str(x)
    x = x.replace('п', '')
    x = x.replace('П', '')
    x = x.replace('nan', '')
    x = x.replace(',', '.')
    return x

df_modes['диаметр'] = df_modes[u'D/S'].apply(split_d)
df_modes['толщина стенки'] = df_modes[u'D/S'].apply(split_s)
df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(clean_d_s)
df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
df_modes['диаметр'] = df_modes['диаметр'].apply(clean_d_s)

In [14]:
# Чистим Скорость прохождения трубы через спрейер, м/с
def clean_u_sp(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u'ек', '')
    x = x.replace(u'\мс', '')
    x = x.replace(u'm/c', '')
    x = x.replace(u'М/С', '')
    x = x.replace(u'М\С', '')
    x = x.replace(u'nan', '')
    x = x.replace(u'м/с', '')
    x = x.replace(u'м\с', '')
    x = x.replace(u',', u'.')
    x = x.replace(u'м\\', '')
    x = x.replace(u'м/', '')
    x = x.replace(u'*****', u'')
    x = x.replace(u'****', u'')
    x = x.replace(u'***', u'')
    x = x.replace(u'29.03.2017', u'')
    x = x.replace(',','.')
    x = x.replace('041','0.41')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')  
    x = x.replace('м','')
    x = x.replace('021', '0.21')
    x = x.replace('1.8', '0.18')
    x = x.replace('2.0', '0.2')
    x = x.replace('012', '0.12')
    if x == '45':
        return '0.45'
    if x == '064':
        return '0.64'
    x = x.replace(u'0.7.', '0.7')
    try:
        y = x.split('-')
        x = (float(y[0])+float(y[1]))/2
    except:
        pass
    return x
    
col = 'Скорость прохождения трубы через спрейер, м/с' 
df_modes[col] = df_modes[col].apply(clean_u_sp)
# df_modes[col].to_excel('check_col.xlsx')

In [15]:
# Чистим время выдержки в закалочной ванне, сек.
def clean_time_bath(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u' ', '')
    x = x.replace(u'.', '')
    return x
col = u'время выдержки в закалочной ванне, сек.'
df_modes[col] = df_modes[col].apply(clean_time_bath)

In [16]:
# Удаляем лишнии столбцы из датафрейма
df_modes[u'№ плавки'] = df_modes[u'плавка']
l2 = [
    'Дата термообработки', u'время начала т/о',u'D/S',u'шаг балок закалочная/отпускная печи, сек', u'плавка',
      '2 зона уставка закалка', '2 зона фактическая температура закалка', '2 зона фактическая температура отпуск', 
      '4 зона фактическая температура отпуск', 'кол-во, шт.']
df_modes = df_modes.drop(l2, axis = 1)
df_modes.shape

(129080, 31)

In [17]:
str_to_del = [3.75, 4.7, 4.18, 4.85, 4.78, 4.56, 3.71, 4.7, 4.63, 4.65, 4.52, 1.57, 3.87, 3.89, 2.5, 1.91, 2.98, 
              4.6, 4.71, 2.36, 4.39, 4.68, 1.82, 2.74, 2.81, 3.61]

for i in str_to_del:
    df_modes.drop(df_modes[df_modes['НД (гост, ту)'] == i].index, inplace=True, axis = 0)

In [18]:
df_modes.shape

(128842, 31)

In [20]:
# df_modes.drop(df_modes[df_modes['№ партии'].isnull()].index, inplace=True, axis = 0)

In [21]:
# df_modes.shape

(128842, 31)

In [22]:
# Объединение марок стали
def rep_steel(x):
    x = str(x)
    x = x.replace('092ГС','09Г2С')
    x = x.replace('13 ХФА','13ХФА')
    x = x.replace('13хфа','13ХФА')
    x = x.replace('13ХФА ','13ХФА')
    x = x.replace('13хфа-3','13ХФА-3')
    x = x.replace('15Х13H2MБ','15Х13H2МБ')
    x = x.replace('15х5мфбч','15Х5МФБЧ')
    x = x.replace('18х3мфб','18Х3МФБ')
    x = x.replace('18Х3МФБ ','18Х3МФБ')
    x = x.replace('18х3ХМФБ','18Х3МФБ')
    x = x.replace('18хзмфб','18Х3МФБ')
    x = x.replace('18хмфб','18ХМФБ')
    x = x.replace('18ХМФБ ','18ХМФБ')
    x = x.replace('20А простая','20А-пр')
    x = x.replace('20А-ПР','20А-пр')
    x = x.replace('20 А','20А')    
    x = x.replace('20A','20А') 
    x = x.replace('20А ','20А')
    x = x.replace('20 АУ','20А-У')
    x = x.replace('20АУ','20А-У')
    x = x.replace('30Г2 ','30Г2')
    x = x.replace('30Г2.','30Г2')
    x = x.replace('30г2-2','30Г2-2')
    x = x.replace('30г2ф','30Г2Ф')
    x = x.replace('30г2','30Г2')
    x = x.replace('30хгма','30ХГМА')
    x = x.replace('30ХГМА1','30ХГМА-1')
    x = x.replace('30хма','30ХМА')
    x = x.replace('32г1','32Г1')
    x = x.replace('32г2','32Г2')
    x = x.replace('32х1мф-1','32Х1МФ-1')
    x = x.replace('32ха','32ХА')
    x = x.replace('37г2ф','37Г2Ф')
    x = x.replace('37г2Ф','37Г2Ф')
    x = x.replace('37Г2ф','37Г2Ф')
    x = x.replace('37Г2Ф ','37Г2Ф')
    x = x.replace('38Г2С ','38Г2С')
    x = x.replace('38г2с-4','38Г2С-4')
    x = x.replace('38Г2-С-9','38Г2С-9')
    x = x.replace('30Г2 ', '30Г2')
    return x

df_modes['марка стали'] = df_modes['марка стали'].apply(rep_steel)

In [24]:
def rep_vtr(x):
    x = str(x)
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('-','')
    x = x.replace('50/*','')
    x = x.replace('нет','')
    x = x.replace('НЕТ','')
    x = x.replace('н/д','')
    return x

df_modes['1 зона по ВТР закалка'] = df_modes['1 зона по ВТР закалка'].apply(rep_vtr)
df_modes['2 зона по ВТР закалка'] = df_modes['2 зона по ВТР закалка'].apply(rep_vtr)
df_modes['3 зона по ВТР закалка'] = df_modes['3 зона по ВТР закалка'].apply(rep_vtr)
df_modes['1 зона ВТР и уставка отпуск'] = df_modes['1 зона ВТР и уставка отпуск'].apply(rep_vtr)
df_modes['2 зона ВТР и уставка отпуск'] = df_modes['2 зона ВТР и уставка отпуск'].apply(rep_vtr)
df_modes['4 зона ВТР и уставка отпуск'] = df_modes['4 зона ВТР и уставка отпуск'].apply(rep_vtr)

In [30]:
def rep_nd(x):
    x = str(x)
    x = x.replace('н/д','')
    x = x.replace('.0', '')
    x = x.replace('НЕТ', '')
    x = x.replace('***', '')
    return x

df_modes['1 зона уставка закалка'] = df_modes['1 зона уставка закалка'].apply(rep_nd)
df_modes['1 зона фактическая температура закалка'] = df_modes['1 зона фактическая температура закалка'].apply(rep_nd)
df_modes['3 зона уставка закалка'] = df_modes['3 зона уставка закалка'].apply(rep_nd)
df_modes['3 зона фактическая температура закалка'] = df_modes['3 зона фактическая температура закалка'].apply(rep_nd)
df_modes['Температура трубы на выгрузке из печи закалки'] = df_modes['Температура трубы на выгрузке из печи закалки'].apply(rep_nd)
df_modes['Расход воды на закалочный спрейер, м3'] = df_modes['Расход воды на закалочный спрейер, м3'].apply(rep_nd)
df_modes['t˚ C воды в закалочной ванне'] = df_modes['t˚ C воды в закалочной ванне'].apply(rep_nd)
df_modes['1 зона фактическая температура отпуск'] = df_modes['1 зона фактическая температура отпуск'].apply(rep_nd)
df_modes['3 зона ВТР и уставка отпуск'] = df_modes['3 зона ВТР и уставка отпуск'].apply(rep_nd)
df_modes['3 зона фактическая температура отпуск'] = df_modes['3 зона фактическая температура отпуск'].apply(rep_nd)
df_modes['5 зона ВТР и уставка отпуск'] = df_modes['5 зона ВТР и уставка отпуск'].apply(rep_nd)
df_modes['5 зона фактическая температура отпуск'] = df_modes['5 зона фактическая температура отпуск'].apply(rep_nd)
df_modes['Температура трубы на выгрузке из печи отпуска'] = df_modes['Температура трубы на выгрузке из печи отпуска'].apply(rep_nd)

In [31]:
def rep_time(x):
    x = str(x)
    x = x.replace('nan','')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')
    x = x.replace('НОРМ','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('40-50','45')
    x = x.replace('сек','')
    x = x.replace('с','')
    x = x.replace(',','.')
    x = x.replace('11cек', '')
    return x


col ='время выдержки в закалочной ванне, сек.'
df_modes[col] = df_modes[col].apply(rep_time)

In [32]:
def rep_tempr(x):
    x = str(x)
    x = x.replace('******','')
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция', '')
    x = x.replace('\\','/')
    x = x.replace('-','/')
    x = x.replace('<', '')
    x = x.replace('>', '')
    x = x.replace('Доотпуск', 'доотпуск')
    x = x.replace('до', '')
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('БизнесТренд', '')
    x = x.replace('ВОЗДУХ', '')
    x = x.replace('Воздух', '')
    x = x.replace('воэдух', '')
    x = x.replace('высадка', '')
    x = x.replace('закалка', '')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('нет', '')
    x = x.replace('НОРМ', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('воздух', '')
    x = x.replace('61702', '')
    x = x.replace('опытные.','')
    x = x.replace('на е','')
    x = x.replace('норм/ия','')
    x = x.replace(',','.')
    y = x.split('/')
    if len(y)==1:
        return x
    fl = True
    tmpy= []
    for i in y:
        if len(i) == 4:
            y.append(i[0:2])
            y.append(i[2:4])
            y.remove(i)
    for i in y:
        try:
            tmpy.append(float(i))
        except:
            tmpy.append(i)
            fl = False
    if(fl):
        y = tmpy   
        x = np.mean(y)
    if x=='0':
        x=''
    return x

col = 't˚ C трубы после спреера'
# df_modes[col].to_excel('check_col_before.xlsx')
df_modes[col] = df_modes[col].apply(rep_tempr)
df_modes.drop(df_modes[df_modes[col]=='отпуск'].index, inplace = True, axis = 0)
df_modes[col] = df_modes[col].apply(
                lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float)
# df_modes[col].to_excel('check_col.xlsx')

In [33]:
def rep_tempr_bath(x):
    x = str(x)
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('бизнес тренд', '')
    x = x.replace('высадка.', '')
    x = x.replace('высадка', '')
    x = x.replace('ВЫСАДКА', '')
    x = x.replace('Высадка', '')
    x = x.replace('высадка', '') 
    x = x.replace('доотпуск.', 'отпуск')
    x = x.replace('доотпуск', 'отпуск')
    x = x.replace('Доотпуск', 'отпуск')
    x = x.replace('отпуск.', 'отпуск')
    x = x.replace('на отпуск', 'отпуск')
    x = x.replace('в малахи', '')
    x = x.replace('ДООТПУСК', 'отпуск')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('НК 8500', '')
    x = x.replace('НК 9000', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('нормал-ция', '')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция.', '')
    x = x.replace('норм-ция', '')
    x = x.replace('нор-ция', '')
    x = x.replace('нор/ция', '')
    x = x.replace('опытные.', '')
    x = x.replace('ОПЫТНЫЕ', '')
    x = x.replace('повторная', '')
    x = x.replace('повтор', '')
    x = x.replace('ПОВТОР', '')
    x = x.replace('Самара', '')
    x = x.replace('СИТЦ', '')
    x = x.replace('Трэнд', '')
    x = x.replace('ЧПТЗ', '')
    x = x.replace('ЧТПЗ', '')
    x = x.replace('30 СЕК.', '30')
    x = x.replace('2/я зак/ка', '')
    x = x.replace('ТРЕНД', '')
    x = x.replace('ПЛ.Р. Ц42/№10','')
    x = x.replace('Челябинск','')
    x = x.replace('нормал/ция', '')
    if x=='0':
        x=''
    if x=='.':
        x=''
    return x
col = 't˚ C трубы после ванны'
# df_modes[col].to_excel('check_col_before.xlsx')
df_modes[col] = df_modes[col].apply(rep_tempr)
df_modes[col] = df_modes[col].apply(rep_tempr_bath)
df_modes.drop(df_modes[df_modes['t˚ C трубы после ванны']=='отпуск'].index, inplace = True, axis = 0)
df_modes[col] = df_modes[col].apply(
                lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float)
# df_modes[col].to_excel('check_col.xlsx')

In [34]:
# df_modes[df_modes['t˚ C трубы после ванны'] == 'ПЛ.Р. Ц42/№10']

In [37]:
df_modes.shape

(128537, 32)

In [38]:
#186
df_modes.drop(df_modes[df_modes['шаг балок закалочная печь, сек'] == '0'].index, inplace=True, axis=0)
#204
df_modes.drop(df_modes[df_modes['шаг балок отпускная печь, сек'] == '0'].index, inplace=True, axis=0)

In [39]:
df_modes.shape

(126113, 32)

In [40]:
ls_mean = df_modes.columns[4:-3]
list(ls_mean)

['1 зона по ВТР закалка',
 '1 зона уставка закалка',
 '1 зона фактическая температура закалка',
 '2 зона по ВТР закалка',
 '3 зона по ВТР закалка',
 '3 зона уставка закалка',
 '3 зона фактическая температура закалка',
 'Температура трубы на выгрузке из печи закалки',
 'Скорость прохождения трубы через спрейер, м/с',
 't˚ C трубы после спреера',
 'Расход воды на закалочный спрейер, м3',
 'время выдержки в закалочной ванне, сек.',
 't˚ C воды в закалочной ванне',
 't˚ C трубы после ванны',
 '1 зона ВТР и уставка отпуск',
 '1 зона фактическая температура отпуск',
 '2 зона ВТР и уставка отпуск',
 '3 зона ВТР и уставка отпуск',
 '3 зона фактическая температура отпуск',
 '4 зона ВТР и уставка отпуск',
 '5 зона ВТР и уставка отпуск',
 '5 зона фактическая температура отпуск',
 'Температура трубы на выгрузке из печи отпуска',
 'шаг балок закалочная печь, сек',
 'шаг балок отпускная печь, сек']

In [41]:
def get_mean(df):
#     df = df.groupby(['№ плавки', '№ партии']).first()
#     print(df.head())
    for col in ls_mean:
        if col in df:
            print(col)
#             try:
            df[col] = df.groupby(['№ плавки', '№ партии'])[col].transform(
                lambda x: x.apply(
                lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float).mean())
#             except:
#                 print(col)
#                 pass
#             df[col] = df[col].apply(lambda x: str(x) if (x!='nan')&(x!='')&(x!=None)&(x!=' ') else None).astype(float)
            
#     df = df.reset_index()
    return df

In [42]:
df_modes['№ партии'] = df_modes['№ партии'].astype(str).str.lower()
df_modes['№ плавки'] = df_modes['№ плавки'].astype(str).str.lower()

In [43]:
df_modes.shape

(126113, 32)

In [44]:
tmp = get_mean(df_modes)
tmp.shape

1 зона по ВТР закалка
1 зона уставка закалка
1 зона фактическая температура закалка
2 зона по ВТР закалка
3 зона по ВТР закалка
3 зона уставка закалка
3 зона фактическая температура закалка
Температура трубы на выгрузке из печи закалки
Скорость прохождения трубы через спрейер, м/с
t˚ C трубы после спреера
Расход воды на закалочный спрейер, м3
время выдержки в закалочной ванне, сек.
t˚ C воды в закалочной ванне
t˚ C трубы после ванны
1 зона ВТР и уставка отпуск
1 зона фактическая температура отпуск
2 зона ВТР и уставка отпуск
3 зона ВТР и уставка отпуск
3 зона фактическая температура отпуск
4 зона ВТР и уставка отпуск
5 зона ВТР и уставка отпуск
5 зона фактическая температура отпуск
Температура трубы на выгрузке из печи отпуска
шаг балок закалочная печь, сек
шаг балок отпускная печь, сек


(126113, 32)

In [47]:
full_ls = list(df_modes.columns)
full_ls = full_ls[-1:]+full_ls[0:-1]

In [48]:
tmp = tmp[full_ls]
tmp = tmp.drop_duplicates()
tmp.to_excel('for_comparision.xlsx')
tmp.shape

(30241, 31)

In [ ]:
tmp['2 зона по ВТР закалка'] = tmp['2 зона по ВТР закалка'].apply(lambda x: x*10 if x<100 else x)
tmp.shape

In [ ]:
tmp.to_excel('all_modes_clean_2.xlsx')